### 作業目的: 使用樹型模型進行文章分類

本次作業主利用[Amazon Review data中的All Beauty](https://nijianmo.github.io/amazon/index.html)來進行review評價分類(文章分類)

資料中將review分為1,2,3,4,5分，而在這份作業，我們將評論改分為差評價、普通評價、優良評價(1,2-->1差評、3-->2普通評價、4,5-->3優良評價)

### 載入套件

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days/day_28-Tree_Base_model_tool_practice/')
print(os.getcwd())

In [ ]:
import json
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

### 資料前處理
文本資料較為龐大，這裡我們取前10000筆資料來進行作業練習

In [ ]:
!ls

In [ ]:
#load json data
all_reviews = []
###<your code>###
with open('All_Beauty_5.json', 'r') as read_file:
    all_reviews = json.loads(read_file.read()) ##read_file.read()
       
#all_reviews[0]

{'overall': 1.0,

 'verified': True,

 'reviewTime': '02 19, 2015',

 'reviewerID': 'A1V6B6TNIC10QE',

 'asin': '0143026860',

 'reviewerName': 'theodore j bigham',

 'reviewText': 'great',

 'summary': 'One Star',
 
 'unixReviewTime': 1424304000}

In [ ]:
#parse label(overall) and corpus(reviewText)
corpus = []
labels = []

###<your code>###
        
#transform labels: 1,2 --> 1 and 3 --> 2 and 4,5 --> 3

###<your code>###

In [ ]:
#preprocessing data
#remove email address, punctuations, and change line symbol(\n)

###<your code>###

In [ ]:
#split corpus and label into train and test
###<your code>###

len(x_train), len(x_test), len(y_train), len(y_test)

In [ ]:
#change corpus into vector
#you can use tfidf or BoW here

###<your code>###

#transform training and testing corpus into vector form
x_train = ###<your code>###
x_test = ###<your code>###

### 訓練與預測

In [ ]:
#build classification model (decision tree, random forest, or adaboost)
#start training

###<your code>###

In [ ]:
#start inference
y_pred = ###<your code>###

In [ ]:
#calculate accuracy
###<your code>###

In [ ]:
#calculate confusion matrix, precision, recall, and f1-score
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

由上述資訊可以發現, 模型在好評的準確度高(precision, recall都高), 而在差評的部分表現較不理想, 在普通評價的部分大部分跟差評搞混,
同學可以試著學習到的各種方法來提升模型的表現